# Step 0: User Setup - Personal Identity and Team Configuration

This notebook establishes the **user context** for the Brett Blocks cybersecurity intelligence system. It creates:

- **Personal Identity Objects**: Your user account, email address, and identity
- **Team Member Objects**: Identities for your immediate team members
- **User Context Storage**: All objects stored in `/usr/` context (no company setup required)

## ✅ What This Notebook Does

1. **Initialize Environment**: Import required libraries and utility functions
2. **Create Personal Identity**: Generate your user account, email, and identity objects
3. **Setup Team Members**: Create identity objects for your team members
4. **Store in User Context**: Save all objects to user context memory (`/usr/` directory)

## 📁 Context Memory Structure Created

```
context_mem/
└── usr/                     # Single user workspace
    ├── cache_me.json        # Your personal identity objects
    ├── cache_team.json      # Team member identity objects
    └── edges.json           # User-level relationships
```

## 🎯 Prerequisites

- No company context required
- Input data files must exist in the specified paths
- This setup is required before creating company contexts

## A.1 Environment Setup

Initialize the core STIX libraries and Brett Blocks utility functions. This section:

- Installs the `stixorm` library for STIX 2.1 object management
- Imports STIX domain objects (Identity, EmailAddress, UserAccount, etc.)
- Imports OS Threat extensions for enhanced identity contact information
- Sets up logging for debugging and monitoring

In [1]:
# Install required STIX libraries
import sys
!{sys.executable} -m pip install stixorm

# Import core STIX 2.1 objects for cybersecurity intelligence
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle
)

# Import OS Threat extensions for enhanced contact information
from stixorm.module.definitions.os_threat import (
    IdentityContact, EmailContact, SocialMediaContact, ContactNumber
)

# Import processing utilities and result handling
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects

# Initialize import factory for STIX object creation
import_type = import_type_factory.get_all_imports()

# Setup logging for debugging and monitoring
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

print("✅ STIX libraries initialized successfully")
print("✅ OS Threat extensions loaded")
print("✅ Logging configured")


[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ STIX libraries initialized successfully
✅ OS Threat extensions loaded
✅ Logging configured


## A.2 Path Configuration

Configure the Python path to enable relative imports of Brett Blocks utilities. This allows the notebook to:

- Access utility functions from the `Utilities/` directory
- Import block creation and context management functions
- Work with the local file system for input/output operations

In [2]:
# Configure Python path for relative imports
# This enables access to Brett Blocks utility functions
import sys
sys.path.append('../')
import os

print(f"✅ Working directory: {os.getcwd()}")
print("✅ Python path configured for Brett Blocks utilities")

✅ Working directory: c:\projects\brett_blocks\Orchestration
✅ Python path configured for Brett Blocks utilities


## A.3 Import Brett Blocks Utilities and Initialize Data

Import the specialized utility functions for:

- **Context Management**: Creating and saving user context objects
- **STIX Object Creation**: Making identity, user account, and email address objects
- **Data Processing**: Converting between different data formats

Also load the identity data configuration that defines the user and team member information.

In [3]:
import json

# Import Brett Blocks utility functions for context management
from Utilities.local_make_general import invoke_save_user_context_block
from Utilities.local_make_general import invoke_save_team_context_block

# Import utilities for data processing and port emulation
from Utilities.util import emulate_ports, unwind_ports, conv

# Import STIX object creation utilities
from Utilities.local_make_sdo import invoke_make_identity_block
from Utilities.local_make_sco import invoke_make_user_account_block, invoke_make_email_addr_block
from Utilities.local_make_sro import invoke_sro_block

# Configure base paths for input data and results
path_base = "../Block_Families/StixORM/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_attributed.json"

# Define identity data for user and team members
# This configuration specifies the input files and result paths for each identity
# Note: paths are relative to the StixORM directory since utility functions add path_base
identities_data = [
    {
        "who": "me",
        "email": "SCO/Email_Addr/email_addr_TR_user.json",
        "acct": "SCO/User_Account/usr_account_TR_user.json",
        "ident": "SDO/Identity/identity_TR_user.json",
        "results": "step0/me"
    },
    {
        "who": "tea1",
        "email": "SCO/Email_Addr/email_addr_team1.json",
        "acct": "SCO/User_Account/usr_account_team1.json",
        "ident": "SDO/Identity/identity_team1.json",
        "results": "step0/nick1"
    },
    {
        "who": "tea2",
        "email": "SCO/Email_Addr/email_addr_team2.json",
        "acct": "SCO/User_Account/usr_account_team2.json",
        "ident": "SDO/Identity/identity_team2.json",
        "results": "step0/hilda2"
    },
    {
        "who": "tea3",
        "email": "SCO/Email_Addr/email_addr_team3.json",
        "acct": "SCO/User_Account/usr_account_team3.json",
        "ident": "SDO/Identity/identity_team3.json",
        "results": "step0/trusty3"
    }
]

print("✅ Brett Blocks utilities imported successfully")
print(f"✅ Base paths configured:")
print(f"   - Objects: {path_base}")
print(f"   - Results: {results_base}")
print(f"✅ Identity data loaded for {len(identities_data)} identities")
for identity in identities_data:
    print(f"   - {identity['who']}: {identity['results']}")

✅ Brett Blocks utilities imported successfully
✅ Base paths configured:
   - Objects: ../Block_Families/StixORM/
   - Results: ../Orchestration/Results/
✅ Identity data loaded for 4 identities
   - me: step0/me
   - tea1: step0/nick1
   - tea2: step0/hilda2
   - tea3: step0/trusty3


## B. Create Your Personal User Identity

This section creates your personal cybersecurity identity consisting of three linked STIX objects:

1. **User Account**: Your system account information (login, user ID, etc.)
2. **Email Address**: Your primary email address for communications
3. **Identity**: Your personal identity object linking account and email

All objects are stored in the **user context** (`/usr/` directory) which requires no company setup.

### B.1 Select Your User Data

Extract the data paths for your personal identity from the configuration. This identifies:

- Input file paths for your email, account, and identity data
- Output directory for storing generated STIX objects
- File naming conventions for user context storage

In [4]:
# Find and extract your personal identity data configuration
for ident in identities_data:
    if ident["who"] == "me":
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]
        
        print("✅ Personal identity data located:")
        print(f"   - Email config: {email_path}")
        print(f"   - Account config: {acct_path}")
        print(f"   - Identity config: {ident_path}")
        print(f"   - Results directory: {results_path}")
        break

✅ Personal identity data located:
   - Email config: SCO/Email_Addr/email_addr_TR_user.json
   - Account config: SCO/User_Account/usr_account_TR_user.json
   - Identity config: SDO/Identity/identity_TR_user.json
   - Results directory: step0/me


### B.2 Create Your User Account

Generate a STIX 2.1 User Account object containing your system account information. This object includes:

- **User ID**: Unique identifier for your account
- **Account Login**: Your system login name
- **Account Type**: Classification of account (admin, user, service, etc.)
- **Display Name**: Human-readable name for your account

The account is automatically saved to the user context with dual-layer storage (STIX data + UI metadata).

In [5]:
# Create your user account STIX object using the utility function
# This reads the account configuration and generates a valid STIX 2.1 UserAccount object
TR_user_acct = invoke_make_user_account_block(acct_path, results_path)

# Configure context type for user context storage
context_type = {
    "context_type": "me"
}

# Define output paths for the generated user account object
acct_results_obj_path = results_base + results_path + "__usr_acct.json"
acct_results_context_path = results_base + "step0/context/TR__usr_acct_context.json"

# Save the user account to user context memory
# This stores the object in /usr/ directory with dual-layer format (STIX + UI metadata)
result = invoke_save_user_context_block(acct_results_obj_path, acct_results_context_path)

print(f"✅ User account created and saved to user context")
print(f"   - Object file: {acct_results_obj_path}")
print(f"   - Context storage: {acct_results_context_path}")
print(f"   - Storage result: {result}")

{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3",
    "user_id": "79563902",
    "account_login": "tjones",
    "account_type": "soc,",
    "display_name": "Trusty Jones"
}
input data->{'stix_object': {'type': 'user-account', 'spec_version': '2.1', 'id': 'user-account--83658594-537d-5c32-b9f0-137354bd9bc3', 'user_id': '79563902', 'account_login': 'tjones', 'account_type': 'soc,', 'display_name': 'Trusty Jones'}}
✅ User account created and saved to user context
   - Object file: ../Orchestration/Results/step0/me__usr_acct.json
   - Context storage: ../Orchestration/Results/step0/context/TR__usr_acct_context.json
   - Storage result: {'context_result': 'User Directory ./generated/os-triage/context_mem/usr\nOptions context saved -> me\nstix_id -> user-account--83658594-537d-5c32-b9f0-137354bd9bc3'}


### B.3 Create Your Email Address

Generate a STIX 2.1 Email Address object for your primary email. This object:

- **Links to your User Account**: Creates a relationship between email and account
- **Stores Contact Information**: Email address and display name
- **Enables Communication Tracking**: Supports incident response and investigation workflows

The email address object is stored in user context and linked to your user account.

In [6]:
# Create your email address STIX object linked to your user account
# This generates a STIX 2.1 EmailAddress object with proper relationships
TR_email_addr = invoke_make_email_addr_block(email_path, results_path, TR_user_acct)

# Configure context type for user context storage
context_type = {
    "context_type": "me"
}

# Define output paths for the generated email address object
email_results_obj_path = results_base + results_path + "__email.json"
email_results_context_path = results_base + "step0/context/TR__email_addr_context.json"

# Save the email address to user context memory
# This creates the relationship between your email and user account
result = invoke_save_user_context_block(email_results_obj_path, email_results_context_path)

print(f"✅ Email address created and linked to user account")
print(f"   - Object file: {email_results_obj_path}")
print(f"   - Context storage: {email_results_context_path}")
print(f"   - Storage result: {result}")

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3",
    "value": "trusty@example.com",
    "display_name": "Trusty Jones",
    "belongs_to_ref": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3"
}
input data->{'stix_object': {'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3', 'value': 'trusty@example.com', 'display_name': 'Trusty Jones', 'belongs_to_ref': 'user-account--83658594-537d-5c32-b9f0-137354bd9bc3'}}
✅ Email address created and linked to user account
   - Object file: ../Orchestration/Results/step0/me__email.json
   - Context storage: ../Orchestration/Results/step0/context/TR__email_addr_context.json
   - Storage result: {'context_result': 'User Directory ./generated/os-triage/context_mem/usr\nOptions context saved -> me\nstix_id -> email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3'}


### B.4 Create Your Personal Identity

Generate your primary STIX 2.1 Identity object that ties together your user account and email address. This identity:

- **Represents You**: Your primary identity in the cybersecurity intelligence system
- **Links All Your Objects**: Connects user account, email, and future objects
- **Enables Investigation Tracking**: Associates your actions and observations with your identity
- **Supports Role-Based Access**: Defines your capabilities and permissions

This is the central identity object for all your cybersecurity intelligence activities.

In [7]:
# Create your personal identity STIX object linking email and account
# This generates a comprehensive STIX 2.1 Identity object with all relationships
TR_ident = invoke_make_identity_block(ident_path, results_path, email_results=TR_email_addr, acct_results=TR_user_acct)

# Configure context type for user context storage
context_type = {
    "context_type": "me"
}

# Define output paths for the generated identity object
ident_results_obj_path = results_base + results_path + "__ident.json"
ident_results_context_path = results_base + "step0/context/TR__ident_context.json"

# Save your complete identity to user context memory
# This creates your primary identity with all linked objects
result = invoke_save_user_context_block(ident_results_obj_path, ident_results_context_path)

print(f"✅ Personal identity created with complete object relationships")
print(f"   - Object file: {ident_results_obj_path}")
print(f"   - Context storage: {ident_results_context_path}")
print(f"   - Storage result: {result}")
print(f"✅ Your cybersecurity intelligence identity is now active")

type identity->{'identity_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'name': 'Trusty Jones', 'description': 'A Trusty Individual', 'roles': ['soc', 'support'], 'identity_class': 'individual', 'sectors': ['technology']}, 'extensions': {'extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498': {'extension_type': 'property-extension', 'contact_numbers': [], 'email_addresses': [], 'social_media_accounts': [], 'first_name': 'Me', 'last_name': 'Jones', 'middle_name': 'Percival', 'prefix': 'Dr', 'team': 'os-threat'}}, 'sub': {'contact_numbers': [{'contact_number_type': 'work-phone', 'contact_number': '0418-208-368'}], 'email_addresses': [{'digital_contact_type': 'work'}], 'social_media_accounts': [{'digital_contact_type': 'work'}]}},

## B.5 Create Your Team Members

Generate STIX identity objects for your immediate team members. This creates:

- **Team Member Accounts**: User accounts for each team member
- **Team Email Addresses**: Email addresses linked to team accounts
- **Team Identities**: Complete identity objects for team collaboration
- **Team Context Storage**: All team objects stored in user context (`/usr/cache_team.json`)

Team members are stored separately from company users to enable quick access for collaboration and incident response.

In [ ]:
# Process each team member identity configuration
team_count = 0

for ident in identities_data:
    results_paths = {}
    
    # Process only team member identities (those starting with "tea")
    if ident["who"][:3] == "tea":
        team_count += 1
        
        # Extract object paths and results path for this team member
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]
        
        print(f"\n📝 Processing team member {team_count}: {ident['who']}")
        
        # Define context pathways for team member objects
        acct_results_obj_path = results_base + results_path + "__usr_acct.json"
        acct_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "t__usr_acct_context.json"
        email_results_obj_path = results_base + results_path + "__email.json"
        email_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "t__email.json"
        ident_results_obj_path = results_base + results_path + "__ident.json"
        ident_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "t__ident_context.json"
        
        # Create and save team member user account
        user_acct = invoke_make_user_account_block(acct_path, results_path)
        result = invoke_save_team_context_block(acct_results_obj_path, acct_results_context_path)
        print(f"   ✅ User account created and saved: {result}")
        
        # Create and save team member email address
        user_email_addr = invoke_make_email_addr_block(email_path, results_path, user_acct)
        result = invoke_save_team_context_block(email_results_obj_path, email_results_context_path)
        print(f"   ✅ Email address created and saved: {result}")
        
        # Create and save team member identity
        user_ident = invoke_make_identity_block(ident_path, results_path, email_results=user_email_addr, acct_results=user_acct)
        result = invoke_save_team_context_block(ident_results_obj_path, ident_results_context_path)
        print(f"   ✅ Identity created and saved: {result}")

print(f"\n🎉 Successfully created {team_count} team member identities")
print(f"✅ All team members stored in user context (/usr/cache_team.json)")
print(f"✅ Team collaboration identities are now active")


📝 Processing team member 1: tea1
{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--c58f2f3c-c3df-5f1a-9f19-e642a01bced3",
    "user_id": "79563903",
    "account_login": "hexpert",
    "account_type": "soc,",
    "display_name": "Helpful Expert"
}
input data->{'stix_object': {'type': 'user-account', 'spec_version': '2.1', 'id': 'user-account--c58f2f3c-c3df-5f1a-9f19-e642a01bced3', 'user_id': '79563903', 'account_login': 'hexpert', 'account_type': 'soc,', 'display_name': 'Helpful Expert'}}
   ✅ User account created and saved: {'context_result': 'Team Directory ./generated/os-triage/context_mem/usr\nOptions context saved -> team\nstix_id -> user-account--c58f2f3c-c3df-5f1a-9f19-e642a01bced3'}
{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--f4889f76-152d-5527-a85f-9e8c07bf71b9",
    "value": "helpful@example.com",
    "display_name": "Helpful Expert",
    "belongs_to_ref": "user-account--c58f2f3c-c3df-5f1a-9f19-e642a01bced3"
}

: 

## 🎉 User Setup Complete!

Your personal user context has been successfully established with:

### ✅ Created Objects
- **Your Personal Identity**: Complete STIX identity with linked account and email
- **Team Member Identities**: All team members ready for collaboration
- **User Context Storage**: All objects stored in `/usr/` directory

### 📁 Context Memory Structure
```
context_mem/
└── usr/
    ├── cache_me.json        # Your personal identity objects
    ├── cache_team.json      # Team member identity objects  
    └── edges.json           # User-level relationships
```

### 🚀 Next Steps
You're now ready to:
1. **Run Step_1_Company_Setup.ipynb** to create company contexts
2. **Create incident investigations** using your established identity
3. **Collaborate with team members** through the team context
4. **Begin cybersecurity intelligence workflows** with your active identity

Your user context will persist across all Brett Blocks sessions and provides the foundation for all cybersecurity intelligence activities.